<a href="https://colab.research.google.com/github/reesha-rsh/MLb4/blob/main/Homework/HW9_Boosting_XGBoost%2C_LightGBM%2C_CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Homework:

- use the dataset and validation approach which you used for previous homeworks
- train xgboost, lightgbm, and catboost models and find the best hyperparameters for each algorithm
- compare results between them and previously trained random forest

# Init

In [1]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import make_scorer, fbeta_score, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import pandas as pd
import numpy as np


from google.colab import drive
drive.mount('/content/drive')

train_full = pd.read_csv("/content/drive/MyDrive/MLb4/EDA Titanic/train.csv")
test_full = pd.read_csv("/content/drive/MyDrive/MLb4/EDA Titanic/test.csv")

Mounted at /content/drive


In [2]:
random_state = 42

In [3]:
def generate(df,age_median,fare_median):
  useless_features = ['Name','Ticket','Cabin']
  data_cleaned = df
  data_cleaned = data_cleaned.drop(columns = useless_features)

  # generate binary values using get_dummies
  data_cleaned = pd.get_dummies(data_cleaned, columns=['Sex'],prefix=["Sex"])
  data_cleaned = pd.get_dummies(data_cleaned, columns=['Embarked'],prefix=["Embarked"])

  # Check for NaN values in the DataFrame
  nan_mask = data_cleaned.isnull()
  # Count the number of NaN values in each column
  nan_count_per_column = data_cleaned.isnull().sum()

  data_cleaned['Age'] = data_cleaned['Age'].fillna(age_median)
  data_cleaned['Fare'] = data_cleaned['Fare'].fillna(fare_median)

  return data_cleaned


In [4]:
not_features = ['PassengerId', 'Survived']

In [5]:
# get medians that will fill NaNs in generate func
age_median = train_full['Age'].median()
print(age_median)
fare_median = train_full['Fare'].median()
print(fare_median)

28.0
14.4542


In [6]:
train = generate(train_full,age_median=age_median,fare_median=fare_median)
print(train)

X_train = train.drop(not_features, axis = 1)
y_train = train['Survived']

     PassengerId  Survived  Pclass   Age  SibSp  Parch     Fare  Sex_female  \
0              1         0       3  22.0      1      0   7.2500           0   
1              2         1       1  38.0      1      0  71.2833           1   
2              3         1       3  26.0      0      0   7.9250           1   
3              4         1       1  35.0      1      0  53.1000           1   
4              5         0       3  35.0      0      0   8.0500           0   
..           ...       ...     ...   ...    ...    ...      ...         ...   
886          887         0       2  27.0      0      0  13.0000           0   
887          888         1       1  19.0      0      0  30.0000           1   
888          889         0       3  28.0      1      2  23.4500           1   
889          890         1       1  26.0      0      0  30.0000           0   
890          891         0       3  32.0      0      0   7.7500           0   

     Sex_male  Embarked_C  Embarked_Q  Embarked_S  

In [7]:
test = generate(test_full,age_median=age_median,fare_median=fare_median)
X_test = test.drop('PassengerId' , axis = 1)

In [8]:
# Initialize the StratifiedKFold object
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

In [9]:
# Define the custom scoring function with the desired beta value
beta = 0.5
custom_scorer = make_scorer(fbeta_score, beta=beta)

#XGBoost

In [10]:
import xgboost as xgb

In [15]:
xgb_train = xgb.DMatrix(X_train, y_train, feature_names=X_train.columns)

num_rounds = 10000
param = {
    #default
    "objective": "binary:logistic",
    "eta": 0.01,
    "verbosity": 0,
    "nthread": 10,
    "random_seed": random_state,
    "feval": custom_scorer,


    # regularization parameters
    "max_leaves": 16,
    "subsample": 0.7,
    "colsample_bytree": 0.9,
    "gamma": 0,
    "min_child_weight": 5,


    #lightgbm approach
    "tree_method": "hist",
    "grow_policy": "lossguide"
}

results = xgb.cv(param, xgb_train, num_rounds, early_stopping_rounds=50,folds=skf, verbose_eval=100)

[0]	train-logloss:0.68906+0.00013	test-logloss:0.68935+0.00010
[100]	train-logloss:0.46626+0.00400	test-logloss:0.47945+0.01062
[200]	train-logloss:0.40395+0.00450	test-logloss:0.43058+0.01665
[300]	train-logloss:0.37409+0.00492	test-logloss:0.41286+0.02047
[400]	train-logloss:0.35697+0.00516	test-logloss:0.40710+0.02327
[500]	train-logloss:0.34452+0.00540	test-logloss:0.40435+0.02584
[600]	train-logloss:0.33471+0.00581	test-logloss:0.40283+0.02860
[700]	train-logloss:0.32643+0.00612	test-logloss:0.40101+0.02967
[800]	train-logloss:0.31920+0.00632	test-logloss:0.39998+0.03174
[900]	train-logloss:0.31266+0.00646	test-logloss:0.39878+0.03275
[1000]	train-logloss:0.30672+0.00666	test-logloss:0.39818+0.03344
[1100]	train-logloss:0.30145+0.00673	test-logloss:0.39750+0.03425
[1200]	train-logloss:0.29651+0.00681	test-logloss:0.39664+0.03440
[1245]	train-logloss:0.29443+0.00689	test-logloss:0.39678+0.03457


In [17]:
model = xgb.XGBClassifier(
    #default
    objective= "binary:logistic",
    eta= 0.01,
    verbosity= 0,
    nthread= 10,
    random_seed= random_state,
    feval= custom_scorer,

    n_estimators = 1245,


    # regularization parameters
    max_leaves= 16,
    subsample= 0.7,
    colsample_bytree= 0.9,
    gamma= 0,
    min_child_weight= 5,


    #lightgbm approach
    tree_method= "hist",
    grow_policy= "lossguide"
)

model.fit(X_train, y_train)

fbeta = cross_val_score(model, X_train, y_train, scoring=custom_scorer, cv=skf).mean()
accuracy = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=skf).mean()

print("F-beta Score (beta={}): {:.4f}".format(beta, fbeta))
print("Accuracy: {:.4f}".format(accuracy))

F-beta Score (beta=0.5): 0.8168
Accuracy: 0.8451


In [18]:
y_pred = model.predict(X_test)
submission_data = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': y_pred})
submission_data.to_csv("/content/drive/MyDrive/MLb4/EDA Titanic/submissionXgb.csv", index=False)

**SUBMISSION SCORE:** 0.76315

# LightGBM

In [19]:
import lightgbm as lgb

In [20]:
# Define the F-beta score function
def custom_fbeta_score(y_true,y_pred):
    beta = 0.5  # Set your desired beta value
    y_true_binary = (y_true > 0.5).astype(int)
    return 'f_beta', fbeta_score(y_true_binary, y_pred.get_label(), beta=beta), True

In [21]:
parameters = {
    #default
    "objective": "binary",
    "learning_rate": 0.01,
    "num_threads": 10,
     "metric": "None",
    "seed":  [random_state],

    #regularization
    "colsample_bytree": 0.7,
    "subsample": 0.7,
    "subsample_freq": 1,
    "min_data_in_leaf": 7,
    'force_col_wise': 'true'

}


n_rounds = 10000

lgb_train = lgb.Dataset(X_train, label=y_train, free_raw_data=False)
result = lgb.cv(parameters, lgb_train, n_rounds,feval=custom_fbeta_score, folds=skf, callbacks=([lgb.early_stopping(stopping_rounds=20),lgb.log_evaluation(period=10, show_stdv=True)]), eval_train_metric=True)

[LightGBM] [Info] Number of positive: 273, number of negative: 439
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 10
[LightGBM] [Info] Number of positive: 274, number of negative: 439
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 713, number of used features: 10
[LightGBM] [Info] Number of positive: 274, number of negative: 439
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 713, number of used features: 10
[LightGBM] [Info] Number of positive: 274, number of negative: 439
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 713, number of used features: 10
[LightGBM] [Info] Number of positive: 273, number of negative: 440
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 713, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pa

In [22]:
lgb_model = lgb.LGBMClassifier(
    objective= "binary",
    learning_rate= 0.01,
    num_threads= 10,
    metric= "None",
    seed=  random_state,
    colsample_bytree= 0.7,
    subsample= 0.9,
    subsample_freq= 1,
    min_data_in_leaf= 7,
    force_col_wise= True,
    n_estimators  = 125)


lgb_model.fit(X_train, y_train,eval_metric =custom_fbeta_score)

fbeta = cross_val_score(lgb_model, X_train, y_train, scoring=custom_scorer, cv=skf).mean()
accuracy = cross_val_score(lgb_model, X_train, y_train, scoring='accuracy', cv=skf).mean()

print("F-beta Score (beta={}): {:.4f}".format(beta, fbeta))
print("Accuracy: {:.4f}".format(accuracy))

y_pred = lgb_model.predict(X_test)
submission_data = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': y_pred})
submission_data.to_csv("/content/drive/MyDrive/MLb4/EDA Titanic/submissionLgb.csv", index=False)


[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Info] Number of positive: 342, number of negative: 549
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 891, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383838 -> initscore=-0.473288
[LightGBM] [Info] Start training from score -0.473288
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_sa

**SUBMISSION SCORE:** 0.77033

# CatBoost

In [23]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.5 MB/s eta 0:00:00


In [24]:
categorical_features_indices = np.where(X_train.dtypes != float)[0]

In [25]:
class FBetaMetric:

    def __init__(self, beta):
        self.beta = beta

    @staticmethod
    def get_fbeta(y_true, y_pred, beta):
        y_pred_binary = (y_pred >= 0.5).astype(int)
        return fbeta_score(y_true, y_pred_binary, beta=beta)

    def is_max_optimal(self):
        return True  # greater is better

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        y_true = np.array(target).astype(int)
        approx = np.array(approxes[0])
        score = self.get_fbeta(y_true, approx, self.beta)
        return score, 1

    def get_final_error(self, error, weight):
        return error

custom_fbeta_metric = FBetaMetric(beta=beta)



In [26]:
import catboost as ctb
parameters = {
    "loss_function": "Logloss",
    "eval_metric": custom_fbeta_metric,
    "iterations": 10000,
    "learning_rate": 0.01,
    "random_seed": random_state,
    "od_wait": 20,
    "od_type": "Iter",
    "thread_count": 10,
    "subsample": 0.7,
    "colsample_bylevel": 0.9
}

ctb_data = ctb.Pool(X_train,y_train,cat_features=categorical_features_indices)
result = ctb.cv(ctb_data, parameters, folds=skf, seed=random_state, verbose_eval=20, plot=False)

/usr/local/lib/python3.10/dist-packages/catboost/core.py:6655: UserWarning: Can't optimze method "evaluate" because self argument is used
  return _cv(params, pool, fold_count, inverted, partition_random_seed, shuffle, stratified,


Training on fold [0/5]
0:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 2.75s	remaining: 7h 38m 31s
20:	learn: 0.6097561	test: 0.6369427	best: 0.6369427 (20)	total: 3.38s	remaining: 26m 47s
40:	learn: 0.7911001	test: 0.7746479	best: 0.7746479 (25)	total: 3.92s	remaining: 15m 51s

bestTest = 0.7746478873
bestIteration = 25

Training on fold [1/5]
0:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 45.7ms	remaining: 7m 37s
20:	learn: 0.6484642	test: 0.6250000	best: 0.6250000 (20)	total: 693ms	remaining: 5m 29s
40:	learn: 0.7957245	test: 0.7500000	best: 0.7670455 (25)	total: 1.51s	remaining: 6m 7s

bestTest = 0.7670454545
bestIteration = 25

Training on fold [2/5]
0:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 35.5ms	remaining: 5m 55s
20:	learn: 0.7227139	test: 0.6845238	best: 0.6845238 (20)	total: 352ms	remaining: 2m 47s
40:	learn: 0.7951807	test: 0.7552083	best: 0.7552083 (23)	total: 797ms	remaining: 3m 13s

bestTest = 0.7552083333
bestIteration 

In [35]:
clf = ctb.CatBoostClassifier(
    loss_function = "Logloss",
    eval_metric = custom_fbeta_metric,
    iterations= 26,
    learning_rate= 0.01,
    random_seed= random_state,
    od_wait = 20,
    od_type= "Iter",
    thread_count= 10,
    subsample= 0.7,
    colsample_bylevel= 0.9)


clf.fit(X_train, y_train,cat_features=categorical_features_indices)

/usr/local/lib/python3.10/dist-packages/catboost/core.py:2268: UserWarning: Can't optimze method "evaluate" because self argument is used
  _check_train_params(params)


0:	learn: 0.0000000	total: 87.5ms	remaining: 2.19s
1:	learn: 0.0000000	total: 97.1ms	remaining: 1.17s
2:	learn: 0.0000000	total: 105ms	remaining: 808ms
3:	learn: 0.0000000	total: 113ms	remaining: 624ms
4:	learn: 0.0000000	total: 122ms	remaining: 513ms
5:	learn: 0.0000000	total: 129ms	remaining: 430ms
6:	learn: 0.0000000	total: 136ms	remaining: 369ms
7:	learn: 0.0000000	total: 144ms	remaining: 323ms
8:	learn: 0.0000000	total: 152ms	remaining: 288ms
9:	learn: 0.0000000	total: 161ms	remaining: 257ms
10:	learn: 0.0000000	total: 171ms	remaining: 234ms
11:	learn: 0.0000000	total: 180ms	remaining: 210ms
12:	learn: 0.0000000	total: 186ms	remaining: 186ms
13:	learn: 0.0000000	total: 190ms	remaining: 163ms
14:	learn: 0.0000000	total: 193ms	remaining: 142ms
15:	learn: 0.0000000	total: 198ms	remaining: 124ms
16:	learn: 0.0000000	total: 201ms	remaining: 106ms
17:	learn: 0.0000000	total: 205ms	remaining: 91ms
18:	learn: 0.0000000	total: 211ms	remaining: 77.9ms
19:	learn: 0.0000000	total: 215ms	remai

In [36]:
fbeta = cross_val_score(clf, X_train, y_train, scoring=custom_scorer, cv=skf).mean()
accuracy = cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=skf).mean()

print("F-beta Score (beta={}): {:.4f}".format(beta, fbeta))
print("Accuracy: {:.4f}".format(accuracy))

/usr/local/lib/python3.10/dist-packages/catboost/core.py:2268: UserWarning: Can't optimze method "evaluate" because self argument is used
  _check_train_params(params)
/usr/local/lib/python3.10/dist-packages/catboost/core.py:2268: UserWarning: Can't optimze method "evaluate" because self argument is used
  _check_train_params(params)


0:	learn: 0.0000000	total: 58.8ms	remaining: 1.47s
1:	learn: 0.0000000	total: 62.7ms	remaining: 752ms
2:	learn: 0.0000000	total: 65.6ms	remaining: 503ms
3:	learn: 0.0000000	total: 68.3ms	remaining: 376ms
4:	learn: 0.0000000	total: 70.9ms	remaining: 298ms
5:	learn: 0.0000000	total: 73.1ms	remaining: 244ms
6:	learn: 0.0000000	total: 75.6ms	remaining: 205ms
7:	learn: 0.0000000	total: 77.6ms	remaining: 175ms
8:	learn: 0.0000000	total: 80.2ms	remaining: 151ms
9:	learn: 0.0000000	total: 82.5ms	remaining: 132ms
10:	learn: 0.0000000	total: 84.8ms	remaining: 116ms
11:	learn: 0.0000000	total: 87.3ms	remaining: 102ms
12:	learn: 0.0000000	total: 89.6ms	remaining: 89.6ms
13:	learn: 0.0000000	total: 91.7ms	remaining: 78.6ms
14:	learn: 0.0000000	total: 94.1ms	remaining: 69ms
15:	learn: 0.0000000	total: 96.4ms	remaining: 60.3ms
16:	learn: 0.0000000	total: 98.8ms	remaining: 52.3ms
17:	learn: 0.0000000	total: 101ms	remaining: 44.9ms
18:	learn: 0.0000000	total: 104ms	remaining: 38.1ms
19:	learn: 0.000000

/usr/local/lib/python3.10/dist-packages/catboost/core.py:2268: UserWarning: Can't optimze method "evaluate" because self argument is used
  _check_train_params(params)
/usr/local/lib/python3.10/dist-packages/catboost/core.py:2268: UserWarning: Can't optimze method "evaluate" because self argument is used
  _check_train_params(params)


21:	learn: 0.0000000	total: 138ms	remaining: 25.1ms
22:	learn: 0.0000000	total: 142ms	remaining: 18.6ms
23:	learn: 0.0000000	total: 146ms	remaining: 12.1ms
24:	learn: 0.0000000	total: 148ms	remaining: 5.92ms
25:	learn: 0.0000000	total: 151ms	remaining: 0us
0:	learn: 0.0000000	total: 58.7ms	remaining: 1.47s
1:	learn: 0.0000000	total: 62ms	remaining: 744ms
2:	learn: 0.0000000	total: 65ms	remaining: 499ms
3:	learn: 0.0000000	total: 67.8ms	remaining: 373ms
4:	learn: 0.0000000	total: 70.4ms	remaining: 296ms
5:	learn: 0.0000000	total: 72.9ms	remaining: 243ms
6:	learn: 0.0000000	total: 75.2ms	remaining: 204ms
7:	learn: 0.0000000	total: 78.6ms	remaining: 177ms
8:	learn: 0.0000000	total: 83.9ms	remaining: 159ms
9:	learn: 0.0000000	total: 87.8ms	remaining: 141ms
10:	learn: 0.0000000	total: 92.5ms	remaining: 126ms
11:	learn: 0.0000000	total: 95.1ms	remaining: 111ms
12:	learn: 0.0000000	total: 97.6ms	remaining: 97.6ms
13:	learn: 0.0000000	total: 99.5ms	remaining: 85.3ms
14:	learn: 0.0000000	total:

/usr/local/lib/python3.10/dist-packages/catboost/core.py:2268: UserWarning: Can't optimze method "evaluate" because self argument is used
  _check_train_params(params)
/usr/local/lib/python3.10/dist-packages/catboost/core.py:2268: UserWarning: Can't optimze method "evaluate" because self argument is used
  _check_train_params(params)


0:	learn: 0.0000000	total: 52.7ms	remaining: 1.32s
1:	learn: 0.0000000	total: 59.8ms	remaining: 718ms
2:	learn: 0.0000000	total: 62.7ms	remaining: 481ms
3:	learn: 0.0000000	total: 65.4ms	remaining: 360ms
4:	learn: 0.0000000	total: 67.9ms	remaining: 285ms
5:	learn: 0.0000000	total: 70.3ms	remaining: 234ms
6:	learn: 0.0000000	total: 72.9ms	remaining: 198ms
7:	learn: 0.0000000	total: 75ms	remaining: 169ms
8:	learn: 0.0000000	total: 77.6ms	remaining: 146ms
9:	learn: 0.0000000	total: 80.1ms	remaining: 128ms
10:	learn: 0.0000000	total: 82.4ms	remaining: 112ms
11:	learn: 0.0000000	total: 84.8ms	remaining: 99ms
12:	learn: 0.0000000	total: 87.4ms	remaining: 87.4ms
13:	learn: 0.0000000	total: 89.6ms	remaining: 76.8ms
14:	learn: 0.0000000	total: 92.2ms	remaining: 67.6ms
15:	learn: 0.0000000	total: 94.6ms	remaining: 59.1ms
16:	learn: 0.0000000	total: 97ms	remaining: 51.4ms
17:	learn: 0.0000000	total: 99.3ms	remaining: 44.1ms
18:	learn: 0.0000000	total: 102ms	remaining: 37.5ms
19:	learn: 0.0000000	

/usr/local/lib/python3.10/dist-packages/catboost/core.py:2268: UserWarning: Can't optimze method "evaluate" because self argument is used
  _check_train_params(params)
/usr/local/lib/python3.10/dist-packages/catboost/core.py:2268: UserWarning: Can't optimze method "evaluate" because self argument is used
  _check_train_params(params)


0:	learn: 0.0000000	total: 55ms	remaining: 1.37s
1:	learn: 0.0000000	total: 59ms	remaining: 708ms
2:	learn: 0.0000000	total: 61.4ms	remaining: 471ms
3:	learn: 0.0000000	total: 63.7ms	remaining: 350ms
4:	learn: 0.0000000	total: 65.9ms	remaining: 277ms
5:	learn: 0.0000000	total: 67.9ms	remaining: 226ms
6:	learn: 0.0000000	total: 69.9ms	remaining: 190ms
7:	learn: 0.0000000	total: 71.7ms	remaining: 161ms
8:	learn: 0.0000000	total: 74ms	remaining: 140ms
9:	learn: 0.0000000	total: 76.4ms	remaining: 122ms
10:	learn: 0.0000000	total: 78.7ms	remaining: 107ms
11:	learn: 0.0000000	total: 81.2ms	remaining: 94.7ms
12:	learn: 0.0000000	total: 83.6ms	remaining: 83.6ms
13:	learn: 0.0000000	total: 86ms	remaining: 73.7ms
14:	learn: 0.0000000	total: 88.1ms	remaining: 64.6ms
15:	learn: 0.0000000	total: 94.6ms	remaining: 59.1ms
16:	learn: 0.0000000	total: 97.8ms	remaining: 51.8ms
17:	learn: 0.0000000	total: 99.8ms	remaining: 44.4ms
18:	learn: 0.0000000	total: 102ms	remaining: 37.6ms
19:	learn: 0.0000000	to

/usr/local/lib/python3.10/dist-packages/catboost/core.py:2268: UserWarning: Can't optimze method "evaluate" because self argument is used
  _check_train_params(params)
/usr/local/lib/python3.10/dist-packages/catboost/core.py:2268: UserWarning: Can't optimze method "evaluate" because self argument is used
  _check_train_params(params)


20:	learn: 0.0000000	total: 98.5ms	remaining: 23.4ms
21:	learn: 0.0000000	total: 102ms	remaining: 18.6ms
22:	learn: 0.0000000	total: 106ms	remaining: 13.8ms
23:	learn: 0.0000000	total: 108ms	remaining: 8.99ms
24:	learn: 0.0000000	total: 110ms	remaining: 4.39ms
25:	learn: 0.0000000	total: 112ms	remaining: 0us
0:	learn: 0.0000000	total: 50.3ms	remaining: 1.26s
1:	learn: 0.0000000	total: 55.3ms	remaining: 664ms
2:	learn: 0.0000000	total: 59.5ms	remaining: 456ms
3:	learn: 0.0000000	total: 64.7ms	remaining: 356ms
4:	learn: 0.0000000	total: 68.7ms	remaining: 288ms
5:	learn: 0.0000000	total: 72.8ms	remaining: 243ms
6:	learn: 0.0000000	total: 77.2ms	remaining: 210ms
7:	learn: 0.0000000	total: 81.6ms	remaining: 184ms
8:	learn: 0.0000000	total: 84.2ms	remaining: 159ms
9:	learn: 0.0000000	total: 87.1ms	remaining: 139ms
10:	learn: 0.0000000	total: 89.9ms	remaining: 123ms
11:	learn: 0.0000000	total: 93ms	remaining: 108ms
12:	learn: 0.0000000	total: 95.7ms	remaining: 95.7ms
13:	learn: 0.0000000	tota

In [37]:
y_pred = clf.predict(X_test)
submission_data = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': y_pred})
submission_data.to_csv("/content/drive/MyDrive/MLb4/EDA Titanic/submissionCat.csv", index=False)

**SUBMISSION SCORE:** 0.76555

#Conclusion


- **Random forest**
  - Submission Accuracy: 0.76555
  - F-beta Score (beta=0.5): 0.8119
  - Accuracy: 0.8440
- **XGBoost**
  - Submission Accuracy: 0.76315
  - F-beta Score (beta=0.5): 0.8168
  - Accuracy: 0.8451
- **LightGBM**
  - Submission Accuracy: 0.77033
  - F-beta Score (beta=0.5): 0.8170
  - Accuracy: 0.8372
- **CatBoost**:
  - Submission Accuracy: 0.76555
  - F-beta Score (beta=0.5): 0.7691
  - Accuracy: 0.8070

